Esta implementação define uma classe Dataset para lidar com conjuntos de dados tabulares. A classe possui os seguintes atributos:

* X: Uma matriz que armazena as variáveis independentes (também chamadas de características de entrada).
* y: Um vetor que armazena a variável dependente (também chamada de variável de saída ou alvo).
* feature_names: Uma lista com os nomes de cada característica de entrada.
label_name: O nome da característica de saída.
* discrete_values: Um dicionário para armazenar os possíveis valores para cada característica discreta.

A classe possui vários métodos para ler e escrever arquivos CSV/TSV, calcular estatísticas e lidar com missing values:

* read_csv: Lê um arquivo CSV/TSV e preenche os atributos X, y, feature_names e label_name.

* write_csv: Escreve o conjunto de dados num arquivo CSV/TSV, incluindo os nomes das características e do rótulo (se disponível).

* describe: Calcula estatísticas para cada característica em X, como médias, desvios padrão, valores mínimos e máximos.

* find_missing_values: Identifica missing values no conjunto de dados e retorna uma matriz booleana com a mesma forma que X, onde True indica um missing value.

* count_missing_values: Conta o número de missing values em cada coluna das características.

* replace_missing_values: Substitui os missing values no conjunto de dados por um valor constante especificado ou pela média/mediana da coluna da característica correspondente.

A classe também possui métodos "getter" e "setter" para aceder e modificar os atributos.

In [49]:
import numpy as np
import csv

class Dataset:
    def __init__(self, X=None, y=None, feature_names=None, label_name=None):

        """
        Inicializa um objeto da classe Dataset.

        Parâmetros:
        - X (numpy array, opcional): Matriz com as variáveis independentes.
        - y (numpy array, opcional): Vetor com a variável dependente.
        - feature_names (list, opcional): Lista com os nomes das características de entrada.
        - label_name (str, opcional): Nome da característica de saída.
        """

        self.X = X
        self.y = y
        self.feature_names = feature_names
        self.label_name = label_name
        self.discrete_values = {}

    def get_X(self):
        """Retorna a matriz X com as variáveis independentes."""
        return self.X

    def get_y(self):
        """Retorna o vetor y com a variável dependente."""
        return self.y

    def get_feature_names(self):
        """Retorna a lista com os nomes das características de entrada."""
        return self.feature_names

    def get_label_name(self):
        """Retorna o nome da característica de saída."""
        return self.label_name

    def set_X(self, X):
        """
        Define a matriz X com as variáveis independentes.

        Parâmetro:
        - X (numpy array): Matriz com as variáveis independentes.
        """
        self.X = X

    def set_y(self, y):
        """
        Define o vetor y com a variável dependente.

        Parâmetro:
        - y (numpy array): Vetor com a variável dependente.
        """
        self.y = y

    def set_feature_names(self, feature_names):
        """
        Define a lista com os nomes das características de entrada.

        Parâmetro:
        - feature_names (list): Lista com os nomes das características de entrada.
        """
        self.feature_names = feature_names

    def set_label_name(self, label_name):
        """
        Define o nome da característica de saída.

        Parâmetro:
        - label_name (str): Nome da característica de saída.
        """
        self.label_name = label_name

    def read_csv(self, filepath, delimiter=',', has_header=True):
        """
        Lê um arquivo CSV/TSV e preenche os atributos X, y, feature_names e label_name.

        Parâmetros:
        - filepath (str): Caminho do arquivo CSV/TSV.
        - delimiter (str, opcional): Delimitador usado no arquivo CSV/TSV. Padrão é ','.
        - has_header (bool, opcional): Indica se o arquivo CSV/TSV possui cabeçalho. Padrão é True.
        """
        with open(filepath, 'r') as file:
            reader = csv.reader(file, delimiter=delimiter)
            if has_header:
                header = next(reader)
                self.feature_names = header[:-1]
                self.label_name = header[-1]
            data = list(reader)
        data = np.array(data, dtype=object)
        self.X = data[:, :-1].astype(np.float64)
        self.y = np.round(data[:, -1].astype(np.float64)).astype(np.int64)

    def write_csv(self, filepath, delimiter=','):
        """
        Escreve o conjunto de dados num arquivo CSV/TSV, incluindo os nomes das características e do rótulo (se disponível).

        Parâmetros:
        - filepath (str): Caminho do arquivo CSV/TSV.
        - delimiter (str, opcional): Delimitador usado no arquivo CSV/TSV. Padrão é ','."""
        data = np.column_stack((self.X, self.y))
        with open(filepath, 'w') as file:
            writer = csv.writer(file, delimiter=delimiter)
            if self.feature_names is not None and self.label_name is not None:
                writer.writerow(self.feature_names + [self.label_name])
            writer.writerows(data)

    def describe(self):
        """
        Calcula e retorna as estatísticas do conjunto de dados (médias, desvios padrão, valores mínimos e máximos).

        Retorna:
        - means (numpy array): Médias das variáveis independentes.
        - stds (numpy array): Desvios padrão das variáveis independentes.
        - min_values (numpy array): Valores mínimos das variáveis independentes.
        - max_values (numpy array): Valores máximos das variáveis independentes.
        """
        means = np.mean(self.X.astype(np.float64), axis=0)
        stds = np.std(self.X.astype(np.float64), axis=0)
        min_values = np.min(self.X.astype(np.float64), axis=0)
        max_values = np.max(self.X.astype(np.float64), axis=0)

        return means, stds, min_values, max_values

    def find_missing_values(self):
        """
        Encontra missing values no conjunto de dados.

        Retorna:
        - missing_values (numpy array): Matriz booleana indicando a presença de missing values.
        """
        return np.isnan(self.X.astype(np.float64))

    def count_missing_values(self):
        """
        Conta a quantidade de missing values no conjunto de dados.

        Retorna:
        - missing_count (numpy array): Vetor com a quantidade de missing values por variável independente.
        """
        return np.sum(np.isnan(self.X.astype(np.float64)), axis=0)

    def replace_missing_values(self, constant='mean'):
        """
        Substitui os missing values no conjunto de dados usando uma constante fornecida, a média ou a mediana.

        Parâmetro:
        - constant (str ou numérico, opcional): Valor para substituir os missing values. Pode ser 'mean', 'median' ou um valor numérico. Padrão é 'mean'.
        """
        missing = self.find_missing_values()
        for i in range(self.X.shape[1]):
            if constant == 'mean':
                replacement = np.nanmean(self.X[:, i].astype(np.float64))
            elif constant == 'median':
                replacement = np.nanmedian(self.X[:, i].astype(np.float64))
            else:
                replacement = constant
            self.X[:, i][missing[:, i]] = replacement

# Testes e exemplos para o algoritmo implementado

## Exemplo de uso do código

Esta implementação demonstra como manipular e analisar um conjunto de dados utilizando a classe Dataset, além de mostrar como salvar e carregar os dados em formato CSV.

In [50]:
from sklearn import datasets

# Carrega o conjunto de dados iris
iris = datasets.load_iris()

# Extrai os nomes das características e o nome do rótulo
feature_names = iris.feature_names
label_name = "species"

# Cria uma instância da classe Dataset com os dados do iris
dataset = Dataset(X=iris.data, y=iris.target, feature_names=feature_names, label_name=label_name)

# Imprime algumas estatísticas
means, stds, min_values, max_values = dataset.describe()
print("Médias:", means)
print("Desvios padrão:", stds)
print("Valores mínimos:", min_values)
print("Valores máximos:", max_values)

# Encontra e conta os missing values
missing_values = dataset.find_missing_values()
print("Valores ausentes:", missing_values)
missing_count = dataset.count_missing_values()
print("Contagem de valores ausentes:", missing_count)

# Escreve o conjunto de dados num arquivo CSV
dataset.write_csv("iris.csv")

# Lê o conjunto de dados do arquivo CSV
dataset_from_csv = Dataset()
dataset_from_csv.read_csv("iris.csv", delimiter=',', has_header=True)

# Imprime algumas estatísticas do conjunto de dados lido do arquivo CSV
means, stds, min_values, max_values = dataset_from_csv.describe()
print("Médias (do CSV):", means)
print("Desvios padrão (do CSV):", stds)
print("Valores mínimos (do CSV):", min_values)
print("Valores máximos (do CSV):", max_values)

Médias: [5.84333333 3.05733333 3.758      1.19933333]
Desvios padrão: [0.82530129 0.43441097 1.75940407 0.75969263]
Valores mínimos: [4.3 2.  1.  0.1]
Valores máximos: [7.9 4.4 6.9 2.5]
Valores ausentes: [[False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False False False]
 [False False

## Testes "Unittest"

Foi realizado um teste utilizando o unittest para testar a implementação do Dataset. Temos então o seguinte teste:

Estes testes cobrem funções como getters e setters, leitura e escrita de arquivos CSV, descrição e manipulação de missing values.

In [51]:
import unittest
import numpy as np
from sklearn import datasets
import os
import tempfile

# Classe que contém os testes unitários para a classe Dataset
class TestDataset(unittest.TestCase):

    # Método executado antes de cada teste
    def setUp(self):
        iris = datasets.load_iris()
        self.dataset = Dataset(X=iris.data, y=iris.target, feature_names=iris.feature_names, label_name="species")

    # Testa os métodos getter e setter da classe Dataset
    def test_getters_and_setters(self):
        X = self.dataset.get_X()
        y = self.dataset.get_y()
        feature_names = self.dataset.get_feature_names()
        label_name = self.dataset.get_label_name()

        self.assertIsNotNone(X)
        self.assertIsNotNone(y)
        self.assertIsNotNone(feature_names)
        self.assertIsNotNone(label_name)

        self.dataset.set_X(np.array([[1, 2], [3, 4]]))
        self.dataset.set_y(np.array([1, 0]))
        self.dataset.set_feature_names(['a', 'b'])
        self.dataset.set_label_name('c')

        self.assertTrue(np.array_equal(self.dataset.get_X(), np.array([[1, 2], [3, 4]])))
        self.assertTrue(np.array_equal(self.dataset.get_y(), np.array([1, 0])))
        self.assertEqual(self.dataset.get_feature_names(), ['a', 'b'])
        self.assertEqual(self.dataset.get_label_name(), 'c')

    # Testa a leitura e escrita de CSVs
    def test_read_write_csv(self):
        self.dataset.write_csv("test_iris.csv")
        dataset_from_csv = Dataset()
        dataset_from_csv.read_csv("test_iris.csv", delimiter=',', has_header=True)

        self.assertTrue(np.array_equal(self.dataset.get_X(), dataset_from_csv.get_X()))
        self.assertTrue(np.array_equal(self.dataset.get_y(), dataset_from_csv.get_y()))
        self.assertEqual(self.dataset.get_feature_names(), dataset_from_csv.get_feature_names())
        self.assertEqual(self.dataset.get_label_name(), dataset_from_csv.get_label_name())

    # Testa a escrita de CSVs usando um arquivo temporário
    def test_write_csv(self):
        # Create a temporary file
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_filepath = temp_file.name

        # Write the dataset to the temporary file
        self.dataset.write_csv(temp_filepath)

        # Read the dataset from the temporary file
        dataset_from_csv = Dataset()
        dataset_from_csv.read_csv(temp_filepath, delimiter=',', has_header=True)

        # Compare the read dataset with the original dataset
        self.assertTrue(np.array_equal(self.dataset.get_X(), dataset_from_csv.get_X()))
        self.assertTrue(np.array_equal(self.dataset.get_y(), dataset_from_csv.get_y()))
        self.assertEqual(self.dataset.get_feature_names(), dataset_from_csv.get_feature_names())
        self.assertEqual(self.dataset.get_label_name(), dataset_from_csv.get_label_name())

        # Remove the temporary file
        os.remove(temp_filepath)

    # Testa a função "describe" da classe Dataset
    def test_describe(self):
        means, stds, min_values, max_values = self.dataset.describe()

        self.assertIsNotNone(means)
        self.assertIsNotNone(stds)
        self.assertIsNotNone(min_values)
        self.assertIsNotNone(max_values)

    # Testa as funções relacionadas aos missing values
    def test_missing_values(self):
        missing_values = self.dataset.find_missing_values()
        self.assertIsNotNone(missing_values)
        self.assertEqual(missing_values.shape, self.dataset.get_X().shape)

        missing_count = self.dataset.count_missing_values()
        self.assertIsNotNone(missing_count)
        self.assertEqual(missing_count.shape, (self.dataset.get_X().shape[1],))

    # Testa a função "replace_missing_values" da classe Dataset
    def test_replace_missing_values(self):
        self.dataset.replace_missing_values(constant='mean')
        missing_values = self.dataset.find_missing_values()
        self.assertFalse(np.any(missing_values))

    # Testa a função "find_missing_values" da classe Dataset
    def test_find_missing_values(self):
        # Create a dataset with some missing values
        X = np.array([[1, np.nan, 3], [4, 5, np.nan], [7, 8, 9]])
        y = np.array([0, 1, 2])
        feature_names = ['a', 'b', 'c']
        label_name = 'd'
        dataset_with_missing_values = Dataset(X=X, y=y, feature_names=feature_names, label_name=label_name)

        # Find missing values
        missing_values = dataset_with_missing_values.find_missing_values()

        # Check if the missing values are correctly identified
        expected_missing_values = np.array([[False, True, False], [False, False, True], [False, False, False]])
        self.assertTrue(np.array_equal(missing_values, expected_missing_values))

    # Testa a função "count_missing_values" da classe Dataset
    def test_count_missing_values(self):
        # Create a dataset with some missing values
        X = np.array([[1, np.nan, 3], [4, 5, np.nan], [7, 8, 9]])
        y = np.array([0, 1, 2])
        feature_names = ['a', 'b', 'c']
        label_name = 'd'
        dataset_with_missing_values = Dataset(X=X, y=y, feature_names=feature_names, label_name=label_name)

        # Count missing values
        missing_count = dataset_with_missing_values.count_missing_values()

        # Check if the count of missing values is correct
        expected_missing_count = np.array([0, 1, 1])
        self.assertTrue(np.array_equal(missing_count, expected_missing_count))

# Executa os testes unitários
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

........
----------------------------------------------------------------------
Ran 8 tests in 0.024s

OK
